In [1]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json 
import requests
import datetime
from datetime import date
pd.set_option('max_colwidth', 800)

In [2]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='a539d7df2c7b43e1ac4d12f386d901e8')

In [3]:
x = newsapi.get_top_headlines(country='us', page_size=50, category = 'health')

In [4]:
nyt_api_key = '13bd501bc77542a58e2e6678619b0d60'

In [5]:
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

# NYT

In [6]:
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))
image_dict = {'keto': 'https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg',
             'ketogenic': 'https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg',
             'paleo': 'https://images.agoramedia.com/everydayhealth/gcms/Can-the-Paleo-Diet-Help-Diabetes-1440x810.jpg',
            'paleolithic': 'https://images.agoramedia.com/everydayhealth/gcms/Can-the-Paleo-Diet-Help-Diabetes-1440x810.jpg',
             'vegan': 'https://familydoctor.org/wp-content/uploads/2011/09/shutterstock_413417941-705x468.jpg',
             'vegetarian': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQo-azo8tn-BcljdjG3wp5Oub2vmC3OvDJ2-EXIhxB5dG3EJAq-'}

def NYT_title_clean(df):
    titles = []
    for index, row in df.iterrows():
        title = row.headline['main']
        titles.append(title)
    df['title'] = titles
    return df

def NYT_dropped_rows(df):
    df = df.drop(['blog','byline','headline','keywords','multimedia','news_desk','print_page','score','type_of_material','uri'],axis =1)
    df.pub_date = pd.to_datetime(df.pub_date).dt.date
    df.word_count = round(df.word_count / 150)
    df.document_type = 'text'
    df['expertise'] = 'Intermediate'
    return df

def NYT_dataframe_clean(df):
    dataframe = NYT_title_clean(df)
    dataframe = NYT_dropped_rows(dataframe)
    return dataframe

def NYT_api_call_section_based(section, source, page, start, end, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:({section_name})&page={page}&source:({source})&begin_date={start}&end_date={end}&api-key={api}'.format(section_name = section, page = page, source = source, start = start, end = end, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = section
    df['image_url'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHEtiVXw8Wi1tp56Nzd5rH_EoOAJA2RInEWvf5h5CQ-6O_YZp7dw'
    return df

def NYT_api_call_parameter_ALLTIME(param, page, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?q={param}&page={page}&sort=newest&&api-key={api}'.format(param = param, page = page, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = param
    df['image_url'] = image_dict[param]
    return df

def diet_NYT():
    empty = pd.DataFrame()
    for word in ['keto','ketogenic','paleo','paleolithic','vegan','vegetarian']:
        df = NYT_api_call_parameter_ALLTIME(word,0,nyt_api_key)
        empty = empty.append(df, sort=True)
        print('Pulled '+word)
        time.sleep(2)
    empty = empty.drop(['abstract','section_name'],axis = 1)
    empty = empty.rename(index=str, columns={"_id": "source_id", "document_type": "medium",'pub_date':'date','snippet':'description','word_count':'length'})
    return empty

# def general_health():
#     empty = pd.DataFrame()
#     for page in [0,1]:
#         df = NYT_api_call_section_based('Health', 'The New York Times',page, last_week, today, nyt_api_key)
#         empty = empty.append(df)
#         time.sleep(2)
#     return empty

def intermediate_search():
    # df1 = general_health()
    df2 = diet_NYT()
    # frames = [df1,df2]
    # result = pd.concat(frames)
    return df2 #result

# Social Media

In [54]:
all_handles = ['@ATPScience1', '@waitrose', '@MicrobiomeInst', '@veganrecipescom', '@cldiet', '@Onnit', '@vegsoc', '@VeganKosher', '@TheVeganSociety', '@vegan', '@Keto_Recipes_', '@the52diet', '@IFdiet', '@microbiome', '@metagenomics', '@microbiome_news', '@TheGutStuff', '@MyGutHealth', '@PaleoFX', '@PaleoFoundation', '@ThePaleoDiet', '@PaleoComfort', '@cavemanketo', '@KetoFlu', '@TheKetoKitchen_', '@EatKetoWithMe', '@KetoConnect', '@KetoDietZone', '@Ketogenic', '@USDANutrition', '@FoodRev', '@CSPI', '@simplyrecipes', '@FoodNetwork', '@CookingChannel', '@tasty', '@nytfood', '@finecooking', '@mrcookingpanda', '@FODMAPeveryday', '@FODMAPLife', '@FodmappedInfo', '@thefodmapdoctor', '@SimplyGlutenFre', '@gfliving', '@sibotest', '@manjulaskitchen', '@VegTimes', '@CookingLight', '@mealprepwl', '@thehealthygut', '@VitalGutHealth', '@pureguthealth', '@PaleoForBegin', '@PaleoLeap', '@ThePaleoMom', '@paleomagazine', '@PaleoHacks', '@paleogrubs', '@naturalgourmet', '@Low_Carb_Keto', '@NutritionTwins', '@mckelhill', '@WomensFitnessAu', '@WomensHealthMag', '@MensHealthMag', '@mjfit', '@thugkitchen', '@Leslie_Klenke', '@insidePN', '@ThisMamaCooks', '@EdibleWildFood', '@TheEarthDieter', '@HarvardHealth', '@EverydayHealth', '@DailyHealthTips']

In [8]:
import tweepy

auth = tweepy.OAuthHandler('UGxquWoEO6bisCK6FGhBlrKYp', 'dRw1GVeyzgdfYQD1S4aEZRGaGCep9QusVZWI9gt4uHriOZG4WI')
auth.set_access_token('333368932-TP9Vp0cWTZ8FywrhtYj3kSvDjhcSPuISON7ROh4f', 'k42e1QG5t72HiDzi4M7WxRfTUvUodZLKFT6268Eb6mLzw')
api = tweepy.API(auth)

In [10]:
test = api.get_user('@DailyHealthTips')

In [86]:
trial = api.user_timeline('@veganrecipescom')

In [87]:
trial

[Status(_api=<tweepy.api.API object at 0x111b82c18>, _json={'created_at': 'Sat Aug 06 07:18:23 +0000 2016', 'id': 761823718398124032, 'id_str': '761823718398124032', 'text': 'Double tap if you want your own veggie garden! Fresh veggies picked from our little veggie… https://t.co/Wpe0YrR3It', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/Wpe0YrR3It', 'expanded_url': 'https://www.instagram.com/p/BIwi8t6gXh3/', 'display_url': 'instagram.com/p/BIwi8t6gXh3/', 'indices': [92, 115]}]}, 'source': '<a href="http://instagram.com" rel="nofollow">Instagram</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2505740822, 'id_str': '2505740822', 'name': 'Zhouer ♡', 'screen_name': 'veganrecipescom', 'location': 'Germany', 'description': "Hi! I'm Zhouer from Vegan Recipes and I make cooking videos on my Yo

In [49]:
trial[0].entities['hashtags'][0]['text']

'AskADoctor'

In [104]:
def clean_tweets(data):
    tweets = []
    for tweet in data:
        try:
            hashtag = tweet.entities['hashtags'][0]['text']
        except IndexError:
            hashtag = 'unknown'
        tweets = [{'title':tweet.id, 'date':tweet.created_at.date().strftime('%Y-%m-%d'), 
           'description': tweet.text, 'source':tweet.user.screen_name,'source_id':tweet.user.id_str,
           'formality': 'Informal'
           ,'length': 1,'medium':'social', 'param':hashtag} for tweet in data]
    tweets = pd.DataFrame(tweets)
    return tweets

In [105]:
def twitter_api_call(list_handles):
    empty = pd.DataFrame()
    for handle in list_handles:
        user_tweets = pd.DataFrame(clean_tweets(api.user_timeline(handle)))
        empty = empty.append(user_tweets, sort=True)
        print(handle)
    empty.title = df.title.astype('str')
    empty['web_url'] = 'https://twitter.com/'+empty.source+'/status/'+empty.title
    empty['image_url'] = empty['web_url']
    return empty

In [96]:
# df = twitter_api_call(all_handles)

In [110]:
df

,date,description,formality,length,medium,param,source,source_id,title,web_url
0,2018-11-12,"The health Guru Ep23 Rhabdomyolysis: What it entails, patterns and outcomes!: https://t.co/2CVCSShb8Y via @YouTube",Informal,1,social,unknown,ATPScience1,575955428,1061862320983470080,https://twitter.com/ATPScience1/status/1061862320983470080
1,2018-11-12,"I added a video to a @YouTube playlist https://t.co/2CVCSShb8Y The health Guru Ep23 Rhabdomyolysis: What it entails, patterns and",Informal,1,social,unknown,ATPScience1,575955428,1061862248128413702,https://twitter.com/ATPScience1/status/1061862248128413702
2,2018-11-09,The ATP Project - Ep183 - Life Hacks and Health Facts: https://t.co/K3krum43Bk via @YouTube,Informal,1,social,unknown,ATPScience1,575955428,1060760971373760512,https://twitter.com/ATPScience1/status/1060760971373760512
3,2018-11-09,Amp-V Chocolate Balls RECIPE: https://t.co/fXTncBCkCx via @YouTube,Informal,1,social,unknown,ATPScience1,575955428,1060753561330573312,https://twitter.com/ATPScience1/status/1060753561330573312
4,2018-11-09,I added a video to a @YouTube playlist https://t.co/fXTncBCkCx Amp-V Chocolate Balls RECIPE,Informal,1,social,unknown,ATPScience1,575955428,1060751360407285760,https://twitter.com/ATPScience1/status/1060751360407285760
5,2018-11-09,Amp-V Chocolate Balls RECIPE: https://t.co/fXTncBkJdX via @YouTube,Informal,1,social,unknown,ATPScience1,575955428,1060751068160655361,https://twitter.com/ATPScience1/status/1060751068160655361
6,2018-11-07,Managing Post Comp Metabolism - Ep182 - The ATP Project: https://t.co/x1sTiaW0gp via @YouTube,Informal,1,social,unknown,ATPScience1,575955428,1060027946944086016,https://twitter.com/ATPScience1/status/1060027946944086016
7,2018-11-02,I added a video to a @YouTube playlist https://t.co/JJcz3fi680 The ATP Project Ep 182 - Managing Post Comp Metabolism,Informal,1,social,unknown,ATPScience1,575955428,1058163652312756225,https://twitter.com/ATPScience1/status/1058163652312756225
8,2018-11-02,Paper of the Week Ep 21 - Idiopathic Hyperprolactinemia Presenting as PCOS: https://t.co/b9mEfhBaPv via @YouTube,Informal,1,social,unknown,ATPScience1,575955428,1058158437471866880,https://twitter.com/ATPScience1/status/1058158437471866880
9,2018-11-02,I added a video to a @YouTube playlist https://t.co/b9mEfhBaPv Paper of the Week Ep 21 - Idiopathic Hyperprolactinemia Presenting as,Informal,1,social,unknown,ATPScience1,575955428,1058158428584067072,https://twitter.com/ATPScience1/status/1058158428584067072


In [101]:
df.title = df.title.astype('str')

In [102]:
df['web_url'] = 'https://twitter.com/'+df.source+'/status/'+df.title

In [108]:
list(df.web_url)[0]

'https://twitter.com/ATPScience1/status/1061862320983470080'

In [109]:
display(Tweet('https://twitter.com/ATPScience1/status/1061862320983470080'))

In [75]:
tweets = [{'title':tweet.id, 'date':tweet.created_at.date().strftime('%Y-%m-%d'), 
           'description': tweet.text, 'source':tweet.user.screen_name,'source_id':tweet.user.id_str,
           'formality': 'Informal'
           ,'length': 1,'medium':'social'} for tweet in trial]

In [76]:
tweets[1]

{'title': 1061974494808629249,
 'date': '2018-11-12',
 'description': 'Largest Health Q&amp;A site. \nAsk a Doctor Online at \nhttps://t.co/kQBzB69Ql9 \n\n#oilyskin #moisturizer… https://t.co/TIsWSmyUo3',
 'source': 'DailyHealthTips',
 'source_id': '16561884',
 'formality': 'Informal',
 'length': 1,
 'medium': 'social',
 'param': 'oilyskin'}

In [ ]:
https://twitter.com/i/web/status/1060826864720248832

In [32]:
from IPython.core.display import HTML, Image

In [ ]:
display(HTML("<a href="+link+">"+source_name+': '+title+"</a>"))

In [29]:
import requests

class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text
